## How would you update illiquid bond price in real time?

### For semi-liquid/illiquid
Bond that occasionally trades, and/or bonds with same issuers/sector/rating that trades

#### Statistical/ML model for bond-pair price difference

- Fit a model on **price adjustments** due to attribute differences: maturity, call schedules, seniorities, etc.
    - For instance, want to price a 7yr nc 3yr bond which has not traded, but a 8yr nc 5yr bond of the same issuer recently trades, then the model should be able to produce the price adjustments from the latter bond to the former.
- The model could be a machine-learning model that does **automatic feature engineering** (e.g. MARS or neural nets), but humans should inspect the results of the feature engineering to make sure it makes sense economically.
    - Alternatively, dealers or traders can suggest **heurestics or functional forms** that determines the parametrics forms of the adjustments.
- As such, the model should train on **pairwise data**: bond price/yield difference of two bonds whose trade timestamps are close.
    - To increase the amount of pairwise data, build a **hierarchy of peer bonds (pricing paths or pricing trees)**, say rating/sector/issuer/seniority.
    - Define the **effective number** for each observation: trades that are more recent, with larger volume and less variation in prices should have higher EN.
    - Then define the **weight of the pair**: pairs with closer trading timestamps, shorter distance in the pricing tree and higher effective numbers on both bonds have higher weights.
        - The intuition of this is that, even bonds of e.g. different issuers can form a pair and be fed to the model, just that these pairs will be given a lower weights than those pairs of the same issuers.
        - We can also impose **restrictions on how pairs are formed**, e.g. just form pairs up to the sector levels, as we believe price adjustments in consumer discretionary are different from TMT.
        - One can even **override pair weights for specific purpose**, for instance, if the model is used to predict coupon adjustments only, then coupon pairs are given more weights.
- To test/CV the model
    - train the model and **predict the near future**
    - do **stratefied hold-out CV**
- To come up with the price of the target bond, one can **weighted average the price adjustments of different pairs**
    - The weights can be just the pair weights aforementioned, or if one wants to get fancy, **train another generative model to produce the weights**
    - The resulting weighted average can **further weighted average with recent trade price** of the target bond.
- Traders/dealers can contribute to this process in at least three ways
    - **add data**: provide quotes where data is extremely lacking
    - **review example inference**: does it make sense that the price difference between these two bonds are like this?
    - **comment on guardrails**

#### Produce bond prices using equity prices

In a nutshell, the idea is to use **structural model** to produce bond prices
- As such, when **equities price ticks**, bond price can be updated in real-time
- Besides equities prices, the key inputs to the model include
    - **recovery**: can be marked by trader
    - **leverage/debt-per-share**: relatively stable across time
    - **asset vol**: can use the implied value $T-1$, given all other inputs and an input bond price
- We cannot just use the classical Merton's model, as that is known to have **difficulties generating for safe names**, one can look at the Credit-Grades model, or Merton's model with jumps.

### For the extremely illiquid bonds
That is, not much bonds that are traded were close enough.

- Build a **supervised learning model**, where $y$ is the price/spread change for the target bond, and $X$ are the returns of macro instruments, such as ETF, single-name CDS, etc. 
- Probably one model per issuer, and by definition, model should be **cross-sectional**: of different bonds.
- Discretization should be at least at **weekly frequency**, and could build-in some **lead-lag relationship**.